# TimesFM 2.5 (PyTorch) による Loto データ予測の動作確認

**目的**:
ローカルの PostgreSQL データベース (`dataset.loto_y_ts`) からデータを取得し、`loto` (ロト種別) および `ts_type` (時系列タイプ) ごとに `y` (値) の将来予測を行います。

**環境**:
- Model: TimesFM 2.5 (200M parameters, PyTorch version)
- Data Source: Local PostgreSQL (`dataset.loto_y_ts`)
- Path: `/mnt/e/env/ts/lib_ana/src/model/timesfm/timesfmV2.ipynb`

**前提条件**:
- 必要なライブラリ (`torch`, `pandas`, `numpy`, `sqlalchemy`, `psycopg2-binary`, `timesfm` 等) がインストールされていること。
- Hugging Face のモデル `google/timesfm-2.5-200m-pytorch` へのアクセスが可能であること。

In [3]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from sqlalchemy import create_engine
from textwrap import dedent

# timesfm モジュールがパスに通っているか確認してください
# 必要であれば sys.path.append() で調整します
# sys.path.append("../../../") 

import timesfm

# ログ設定の抑制（任意）
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

print(f"Python: {sys.version}")
print(f"Torch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

Python: 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
Torch: 2.4.1+cu121
CUDA Available: True


## データベースからのデータ取得

In [4]:
# データベース接続情報の定義
DB_CONFIG = {
    "user": "loto",
    "password": "z",
    "host": "127.0.0.1",
    "port": "5432",
    "database": "loto"
}

# 接続文字列の作成 (PostgreSQL)
db_url = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# エンジンの作成
engine = create_engine(db_url)

# データ取得クエリ
# loto, ts_type ごとに時系列順 (ds) にデータを取得します
query = """
SELECT
    loto,
    ts_type,
    ds,
    y
FROM
    dataset.loto_y_ts
ORDER BY
    loto,
    ts_type,
    ds ASC
"""

try:
    print("Fetching data from database...")
    df = pd.read_sql(query, engine)
    print(f"Data loaded successfully. Shape: {df.shape}")
    display(df.head())
except Exception as e:
    print(f"Error loading data: {e}")

Fetching data from database...
Data loaded successfully. Shape: (456096, 4)


,loto,ts_type,ds,y
0,bingo5,cumsum,2017-04-05,10.0
1,bingo5,cumsum,2017-04-05,19.0
2,bingo5,cumsum,2017-04-05,13.0
3,bingo5,cumsum,2017-04-05,23.0
4,bingo5,cumsum,2017-04-05,1.0


## データのバッチ化とアテンションマスクの作成

ロードしたデータフレーム (`df`) は「ロング形式」であり、系列ごとに長さが異なる可能性があります。
TimesFM などの Transformer モデルに入力するため、以下の前処理を行います。

1.  **グルーピング**: `loto` と `ts_type` の組み合わせごとに時系列データを抽出します。
2.  **左パディング (Left Padding)**: バッチ内の最大系列長に合わせて、各系列の先頭（過去側）を埋めます。これは、因果的マスク(Causal Mask)を適用する時系列モデルで一般的です。
3.  **マスク情報の生成**: 各系列で「どれだけパディングしたか」を `num_all_masked_kv` として計算し、`make_attn_mask` に渡します。

In [5]:
import numpy as np
import jax.numpy as jnp
from timesfm.flax.transformer import make_attn_mask

# 1. データを系列ごとに分割・リスト化
# (loto, ts_type) をキーとして、yの値を時系列順(ds順)に抽出
grouped = df.groupby(['loto', 'ts_type'])
sequences = [group['y'].values for _, group in grouped]

# 2. バッチ内の最大長(max_len)を計算
# これが query_length (入力シーケンス長) になります
max_len = max(len(seq) for seq in sequences)
batch_size = len(sequences)

print(f"Batch size: {batch_size}, Max sequence length: {max_len}")

# 3. 左パディング処理とパディング数の計算
# Transformerへの入力配列 (batch, length) と、パディング数 (batch,) を用意
batched_input = np.zeros((batch_size, max_len), dtype=np.float32)
num_all_masked_kv_list = []

for i, seq in enumerate(sequences):
    seq_len = len(seq)
    # 左パディング数 = 最大長 - 現在の系列長
    pad_len = max_len - seq_len
    num_all_masked_kv_list.append(pad_len)
    
    # 配列の右側（直近）にデータを配置し、左側（過去）は0埋め
    if seq_len > 0:
        batched_input[i, pad_len:] = seq

# JAX配列(DeviceArray)へ変換
# num_all_masked_kv は int32 である必要があります
num_all_masked_kv = jnp.array(num_all_masked_kv_list, dtype=jnp.int32)
batched_input_jax = jnp.array(batched_input) # ※実際のモデル入力用

# 4. アテンションマスクの作成
# query_length は Pythonの int (static) で渡す必要があります
result_mask = make_attn_mask(
    query_length=int(max_len),          # バッチ内の最大系列長
    num_all_masked_kv=num_all_masked_kv, # 各バッチの左パディング数
    query_index_offset=None,            # 通常推論(キャッシュなし)ならNoneでOK
    kv_length=0,                        # 0なら自己注意 (kv_length=query_length)
)

print("-" * 30)
print(f"Mask shape: {result_mask.shape}") # (Batch, 1, Query, Key)
print(f"Sample mask (first batch item):\n{result_mask[0, 0, :, :]}")

Batch size: 36, Max sequence length: 27664
------------------------------
Mask shape: (36, 1, 27664, 27664)
Sample mask (first batch item):
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ...  True False False]
 [False False False ...  True  True False]
 [False False False ...  True  True  True]]


## RevIN (Reverse Instance Normalization) の適用

モデルの学習安定性と予測精度を向上させるため、入力データに対して正規化を行います。
RevINでは、入力系列ごとの平均($\mu$)と標準偏差($\sigma$)を用いて正規化し、推論後の出力に対して逆変換（非正規化）を行うことで元のスケールに戻します。

ここでは、入力データ `batched_input_jax` から統計量を算出し、正規化を実行します。

In [6]:
import jax.numpy as jnp
from timesfm.flax.util import revin

# 1. 統計量（平均と標準偏差）の計算
# 時間軸 (axis=1) に沿って計算し、ブロードキャスト用に次元を保持 (keepdims=True) します。
# input shape: [Batch, Time] -> stats shape: [Batch, 1]
mu = jnp.mean(batched_input_jax, axis=1, keepdims=True)
sigma = jnp.std(batched_input_jax, axis=1, keepdims=True)

# ゼロ除算を防ぐための微小値加算（一般的に行われる処理ですが、データにより調整）
eps = 1e-5
sigma = jnp.maximum(sigma, eps)

# 2. 正規化 (Normalization) の実行
# TODO の箇所を埋めています
normalized_input = revin(
    x=batched_input_jax,  # Float[Array, 'batch time']
    mu=mu,                # Float[Array, 'batch 1']
    sigma=sigma,          # Float[Array, 'batch 1']
    reverse=False,        # False = 正規化 (Input -> Normalized)
)

print(f"Original stats - Mean: {mu[0,0]:.4f}, Std: {sigma[0,0]:.4f}")
print(f"Normalized input shape: {normalized_input.shape}")
print(f"Normalized sample (first 5): {normalized_input[0, :5]}")

Original stats - Mean: 618.1191, Std: 2163.3667
Normalized input shape: (36, 27664)
Normalized sample (first 5): [-0.2857209 -0.2857209 -0.2857209 -0.2857209 -0.2857209]


## スキャン処理 (Sequential Processing)

時系列データの生成や状態更新を行うため、`scan_along_axis` を使用して時間軸（axis=1）に沿ったループ処理を行います。
これは JAX の `jax.lax.scan` のラッパーであり、Python の `for` ループよりも高速にコンパイルされます。

* **`f` (step function)**: 各ステップで実行される関数。`(carry, x) -> (new_carry, y)` の形式を取ります。
* **`init`**: ループの初期状態（carryの初期値）。KVキャッシュの初期状態などが該当します。
* **`xs`**: スキャン対象の入力シーケンス。
* **`axis`**: ループを回す軸（ここでは時間軸 `1`）。

以下の例では、仕組みを理解するために単純な「累積和（Cumulative Sum）」を計算しています。

In [8]:
import jax
import jax.numpy as jnp
from timesfm.flax.util import scan_along_axis

# 1. ステップ関数の定義
# 実際のTimesFMでは、ここはモデルの推論ステップ (decode_step) になります
def mock_step_fn(carry, x):
    """
    carry: 前のステップから持ち越された状態 (例: 累積値, KV Cache)
    x: 現在のステップの入力
    """
    new_carry = carry + x       # 状態の更新 (ここでは累積和)
    output = new_carry * 1.0    # 現在のステップの出力
    return new_carry, output

# 2. ダミーデータの準備
# [Batch, Time, Feature]
B, T, D = 2, 5, 1
xs_dummy = jnp.ones((B, T, D), dtype=jnp.float32)  # 入力: 全て1
init_state = jnp.zeros((B, D), dtype=jnp.float32)  # 初期状態: 0

# 3. scan_along_axis の実行
# TODO の箇所を埋めています
scan_result = scan_along_axis(
    f=mock_step_fn,      # 各ステップで実行する関数
    init=init_state,     # 初期状態 (carry)
    xs=xs_dummy,         # 入力シーケンス
    axis=1,              # 時間軸に沿ってスキャン
    # kwargs={}            # step_fn に渡す追加引数があれば辞書で指定
)

# scan_along_axis の戻り値は (final_carry, stacked_outputs)
final_state, outputs = scan_result

print("Input (xs):\n", xs_dummy[0, :, 0])
print("Output (Cumulative Sum):\n", outputs[0, :, 0])
print(f"Output Shape: {outputs.shape}") # (2, 5, 1) - 入力と同じ形状が保たれる

Input (xs):
 [1. 1. 1. 1. 1.]
Output (Cumulative Sum):
 [1. 2. 3. 4. 5.]
Output Shape: (2, 5, 1)


## 統計量更新ロジックの修正

`update_running_stats`関数の戻り値が2つ（`new_mu`, `new_sigma`）である場合に、3つの変数でのアンパッキングを行おうとして`ValueError`が発生していました。
これに対処するため、戻り値の要素数を確認し、2つの場合は`new_n`（サンプル数）をマスクの和から手動で更新するように修正します。これにより、ライブラリの仕様変更や実装の差異に柔軟に対応します。

In [10]:
# ---------------------------------------------------------------------------
# 修正版: 統計量の更新処理
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

# マスクの作成 (Batch, P)
mask_new = jnp.ones((B, P), dtype=jnp.bool_)

# 3. 統計量の更新
# 戻り値の数を確認してアンパッキングを行う
# 通常は (new_n, new_mu, new_sigma) だが、実装により (new_mu, new_sigma) の場合があるため分岐処理を追加

stats_result = update_running_stats(
    n=n_current,
    mu=mu_current,
    sigma=sigma_current,
    x=x_new,
    mask=mask_new,
)

if len(stats_result) == 3:
    # 期待通り3つの値が返された場合
    new_n, new_mu, new_sigma = stats_result
elif len(stats_result) == 2:
    # 2つの値（mu, sigma）のみが返された場合
    new_mu, new_sigma = stats_result
    
    # new_n を手動で更新
    # マスクされている有効なデータ数を現在のカウントに加算
    # axisはデータの次元に合わせて調整（ここでは最終次元の和を想定）
    increment = jnp.sum(mask_new, axis=-1, keepdims=True)
    new_n = n_current + increment
else:
    raise ValueError(f"Unexpected return values from update_running_stats: expected 2 or 3, got {len(stats_result)}")

print("--- Stats Update ---")
print(f"Count: {n_current} -> {new_n}")
# 必要であれば形状や値の確認
# print(f"Mu shape: {new_mu.shape}, Sigma shape: {new_sigma.shape}")

--- Stats Update ---
Count: [10. 10.] -> [[14. 14.]
 [14. 14.]]


## 欠損データの線形補間 (Linear Interpolation)

時系列データに含まれる欠損値（`NaN`: Not a Number / 非数）に対し、**線形補間 (Linear Interpolation)** を適用して値を充填します。

### 1. 処理の目的 (Purpose)
多くの時系列予測モデルは、入力データが連続していることを前提としており、欠損値が含まれていると計算エラー（`NaN`の伝播）や精度の低下を引き起こします。本処理により、データの完全性（Integrity）と連続性を確保します。

### 2. アルゴリズムの仕組み (Mechanism)
線形補間とは、グラフ上でデータが存在する2つの点（欠損の直前と直後）を定規で結ぶように直線を引き、その直線上にある値を推測値として採用する手法です。
* **特徴**: 単純かつ計算コストが低い一方で、データの急激な変動には追従しにくい特性があります。しかし、一般的なトレンド（傾向）を維持する上では堅実な手法です。

### 3. 実装上の注意 (Implementation Note)
* **`arr` (Array / 配列)**: 補間対象の入力データです。`TODO` 部分には、欠損を含む具体的な変数（例: `context_data` や `ts_array` など）を指定してください。

In [11]:
# ---------------------------------------------------------------------------
# 欠損値補間の実行
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

import jax.numpy as jnp
from timesfm.timesfm_2p5.timesfm_2p5_base import linear_interpolation

# 補間対象のデータを定義
# NOTE: 前のセルで生成された 'x_new' (または処理したい時系列配列) を指定してください
# 文脈に基づき、ここでは 'x_new' を対象とします
target_array = x_new 

# 線形補間 (Linear Interpolation) の実行
# arr: 欠損値(NaN)を含む可能性のある入力配列
result_interpolated = linear_interpolation(
    arr=target_array
)

# --- 結果の確認 (Meta-check) ---
print("--- Linear Interpolation Result ---")
print(f"Original shape: {target_array.shape}")
print(f"Interpolated shape: {result_interpolated.shape}")

# 補間前後のNaN数を確認し、処理の有効性(Integrity)を検証
original_nans = jnp.isnan(target_array).sum()
remaining_nans = jnp.isnan(result_interpolated).sum()

print(f"NaN count (Before): {original_nans}")
print(f"NaN count (After) : {remaining_nans}")

if remaining_nans == 0:
    print("Success: All missing values have been interpolated.")
else:
    print("Warning: Some NaN values remain (likely at the edges of the series).")

--- Linear Interpolation Result ---
Original shape: (2, 4)
Interpolated shape: (2, 4)
NaN count (Before): 0
NaN count (After) : 0
Success: All missing values have been interpolated.


## 先頭欠損値の除去 (Stripping Leading NaNs)

時系列配列の先頭（開始部分）に連続して存在する **NaN (Not a Number / 非数)** を除去、またはトリミングする処理を行います。

### 1. 処理の目的 (Purpose)
* **有効データの抽出 (Extraction):** 多くの時系列データでは、実際の計測が始まるまでの期間が `NaN` として埋められている場合があります。線形補間（Linear Interpolation）では「前後の値」が必要なため、この先頭の `NaN` は補間されずに残ることがあります。
* **モデルの安定化 (Stabilization):** 学習や推論において、無意味な `NaN` 列を入力することは、計算エラーや勾配（Gradient）の消失・爆発を引き起こす要因となります。有効な数値が始まる時点をデータの「開始」と定義し直すことで、モデルの挙動を安定させます。

### 2. アルゴリズムの仕組み (Mechanism)
配列の先頭（インデックス0）から順に値をスキャンし、**最初の有効な数値 (Valid Number)** が現れる位置を特定します。その位置より前のデータを切り捨てるか、無視するマスク（Mask）を生成します。
* ※実装によっては、配列自体の長さを短縮する場合と、パディング（Padding）として処理する場合の2パターンがありますが、本関数はライブラリの仕様に従い適切な形式（通常はトリミング後の配列）を返します。

### 3. 実装上の注意 (Implementation Note)
* **`arr`**: 処理対象の配列です。前のステップで線形補間を行った結果（例: `result_interpolated`）を入力することが一般的です。

In [12]:
# ---------------------------------------------------------------------------
# 先頭の欠損値(NaN)除去の実行
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

import jax.numpy as jnp
from timesfm.timesfm_2p5.timesfm_2p5_base import strip_leading_nans

# 処理対象のデータを定義
# NOTE: 前のステップ（線形補間など）の結果変数を指定してください。
# ここでは文脈に基づき 'result_interpolated' を対象とします。
# もし補間を行っていない場合は元の 'x_new' などを指定します。
input_array = result_interpolated 

# 先頭のNaNを除去 (Strip Leading NaNs)
result_stripped = strip_leading_nans(
    arr=input_array
)

# --- 結果の確認 (Meta-check) ---
print("--- Strip Leading NaNs Result ---")
print(f"Original shape: {input_array.shape}")
print(f"Stripped shape: {result_stripped.shape}")

# 先頭の値を確認（有効な数値で始まっているか）
# ※多次元配列の場合は最初のバッチを表示
if result_stripped.ndim > 1:
    first_val = result_stripped[0, 0]
    print(f"First value of first batch: {first_val}")
else:
    first_val = result_stripped[0]
    print(f"First value: {first_val}")

# 検証 (Verification)
if not jnp.isnan(first_val):
    print("Success: The series now starts with a valid number.")
else:
    print("Warning: The series still starts with NaN (Possible all-NaN series).")

--- Strip Leading NaNs Result ---
Original shape: (2, 4)
Stripped shape: (2, 4)
First value of first batch: 1.0
Success: The series now starts with a valid number.


## Transformer層の適用 (Applying Stacked Transformers)

ロードした時系列データに対し、Transformerの主要部分（Attention層とFeedForward層の積み重ね）を適用する準備を行います。

### 1. データ整形プロセス (Data Preparation)
データベースから取得した `df` は「ロング形式（縦持ち）」です。これをモデル入力用に以下の手順で変換します。
1.  **ピボット (Pivot):** `loto`, `ts_type` をインデックス、`ds` をカラムとして展開し、`(Batch, Sequence)` の形状にします。
2.  **埋め込みシミュレーション (Embedding Simulation):** `_apply_stacked_transformers` は、次元 $D$ (Model Dimension) を持つ入力を期待します。ここでは実際のPatch Embeddingの代わりに、次元を拡張して入力をエミュレートします。

[Image of Transformer model architecture input embedding]

### 2. 引数の定義 (Arguments)
* **`x`**: 入力テンソル。形状は `(Batch, Sequence, Model_Dim)` です。
* **`m`**: マスク。パディング部分（データがない部分）を無視するために使用します。今回はすべて有効データとして `1` で埋めます。
* **`model`**: 本来は学習済みのFlaxモジュールですが、ここでは動作確認用にダミー（Mock）を使用します。

In [17]:
# ---------------------------------------------------------------------------
# データ整形とTransformer適用のエミュレーション
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

import jax
import jax.numpy as jnp
import pandas as pd
from unittest.mock import MagicMock
from timesfm.timesfm_2p5.timesfm_2p5_flax import _apply_stacked_transformers

# --- 1. DBデータの整形 (Preprocessing) ---
# ロング形式のDFを (Batch, Time) に変換
if 'df' in locals() and not df.empty:
    # ユニークなIDごとに時系列を横に並べる
    pivot_df = df.pivot_table(index=['loto', 'ts_type'], columns='ds', values='y')
    # 欠損がある場合は0埋めなどを行う（必要に応じて変更）
    pivot_df = pivot_df.fillna(0)
    
    raw_array = jnp.array(pivot_df.values) # Shape: (Batch, Sequence)
    print(f"Pivoted Data Shape: {raw_array.shape}")
else:
    # データがない場合のフォールバック
    raw_array = jnp.zeros((2, 32))

# --- 2. 変数の準備 (Preparation) ---
B, N = raw_array.shape
D = 64  # Model Dimension (本来はモデル設定に依存。例: 512, 1024など)

# x: 入力埋め込み (Input Embeddings)
# 注意: Transformer層への入力は (Batch, Sequence, Model_Dim) である必要があります。
# 実際のパイプラインでは PatchEmbedding 層の出力を使いますが、
# ここでは raw_array を線形射影して擬似的に作成します。
key = jax.random.PRNGKey(42)
projection_matrix = jax.random.normal(key, (1, D)) # (1, D)
# (B, N, 1) * (1, D) -> (B, N, D) へのブロードキャスト的な擬似拡張
x_input = raw_array[..., None] * projection_matrix 

# m: アテンションマスク (Mask)
# すべて有効なデータと仮定して 1.0 (float) または True (bool) を設定
# 実装により型が異なりますが、JAXでは通常 float のマスク(1.0 keep, 0.0 drop)を使うことが多いです
m_input = jnp.ones((B, N), dtype=jnp.float32)

# model: Transformerモデル (Mock)
# 内部で model.layers などを参照するため、ダミーオブジェクトを作成
# ※実際に計算させるには本物のFlax Moduleが必要ですが、ここではエラー回避のためのMockです
mock_model = MagicMock()
mock_model.layers = [] # レイヤーリストを空にしておけばループが回らずそのまま出力される想定

# --- 3. 関数の実行 (Execution) ---
print("--- Applying Stacked Transformers (Mock) ---")
try:
    result = _apply_stacked_transformers(
        model=mock_model, # TODO: 本来は学習済みTransformerモデルインスタンス
        x=x_input,        # TODO: Float[Array, 'b n d']
        m=m_input,        # TODO: Float[Array, 'b n']
        decode_cache=None,
    )
    
    print("Success!")
    print(f"Input shape: {x_input.shape}")
    print(f"Output shape: {result.shape}")

except Exception as e:
    print(f"Simulation skipped due to missing model structure: {e}")
    # 実際のモデルがないと内部処理でエラーになる可能性が高いため、
    # その場合は入力をそのまま出力とするパススルーを提示
    result = x_input
    print("Fallback: Returned input as result for data flow check.")

Pivoted Data Shape: (36, 7366)
--- Applying Stacked Transformers (Mock) ---
Simulation skipped due to missing model structure: Expected an array, got MagicMock args[0]
Fallback: Returned input as result for data flow check.


## モデル入力前の正規化と前処理 (Pre-Model Decode Processing)

生の時系列データ（Raw Input）をモデルが処理しやすい形式に変換します。具体的には、各入力ウィンドウに対して平均 $\mu$ と標準偏差 $\sigma$ を計算し、正規化（Standardization）を行います。

### 1. 処理の目的 (Purpose)
* **スケール不変性 (Scale Invariance):** 株価（数万円）と気温（数十度）のようにスケールが異なるデータでも、モデルが同様にパターンを学習できるようにします。
* **数値安定性 (Numerical Stability):** データを $\mu=0, \sigma=1$ 近傍に分布させることで、勾配消失や発散を防ぎます。
* **統計量の保持 (Statistics Retention):** 計算された $\mu$ と $\sigma$ は、推論後の「非正規化（Denormalization）」ステップで使用するため、ここで保持します。

[Image of time series normalization process]

### 2. 引数の定義 (Arguments)
* **`fc` (Forecast Context):** 予測設定やハイパーパラメータを保持するコンテキストオブジェクト。
* **`inputs`**: モデルへの入力時系列データ $(Batch, Time)$。
* **`masks`**: パディングマスク（有効なデータ＝1、無効＝0）。欠損値や系列長の不足部分を無視させるために使用します。

In [21]:
# ---------------------------------------------------------------------------
# 修正版(v3): 属性を追加したコンテキストによる前処理の実行
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

import jax
import jax.numpy as jnp
from collections import namedtuple
from timesfm.timesfm_2p5.timesfm_2p5_flax import _before_model_decode

# --- 1. 入力データの準備 ---
if 'raw_array' in locals():
    input_tensor = raw_array
else:
    # Batch=2, Time=32 のダミー
    input_tensor = jnp.array([
        jnp.sin(jnp.linspace(0, 10, 32)) + 5,
        jnp.cos(jnp.linspace(0, 10, 32)) * 10
    ])

# マスクの作成
input_masks = jnp.ones_like(input_tensor, dtype=jnp.float32)

# --- 2. コンテキストの作成 (属性追加) ---
# エラーで指摘された 'normalize_inputs' をフィールドに追加します。
MockFC = namedtuple("MockFC", [
    "infer_is_positive", 
    "normalize_inputs"   # 追加: 正規化を実行するかどうかのフラグ
])

# インスタンス化
mock_fc = MockFC(
    infer_is_positive=True,  # 非負値の推論: On
    normalize_inputs=True    # 正規化処理: On
)

# --- 3. 前処理の実行 ---
print("--- Pre-Model Decode Processing (v3) ---")

try:
    # JAX jitコンパイル
    result_tuple = _before_model_decode(
        fc=mock_fc,          
        inputs=input_tensor,
        masks=input_masks
    )

    # --- 結果の確認 ---
    # 戻り値の構成: (normalized_inputs, mu, sigma, is_positive)
    if isinstance(result_tuple, tuple):
        print(f"Result Tuple Length: {len(result_tuple)}")
        
        # 1. 正規化された入力
        norm_inputs = result_tuple[0]
        print(f"Normalized Input Shape: {norm_inputs.shape}")
        
        # 2. 統計量 (mu, sigma)
        if len(result_tuple) >= 3:
            mu = result_tuple[1]
            sigma = result_tuple[2]
            print(f"Mu (Mean) Shape: {mu.shape}")
            print(f"Sigma (Std) Shape: {sigma.shape}")
            
            # 検証: 正規化後の平均は0、分散は1に近い値になるはず
            print(f"Mean (norm): {jnp.mean(norm_inputs):.4f}")
            print(f"Std  (norm): {jnp.std(norm_inputs):.4f}")
            
        # 3. 正値フラグ (is_positive)
        if len(result_tuple) >= 4:
            is_pos = result_tuple[3]
            print(f"Is Positive Flag Shape: {is_pos.shape}")
            print(f"Is Positive (Example): {is_pos[0]}")

    else:
        print(f"Result Type: {type(result_tuple)}")

except AttributeError as e:
    print(f"AttributeError: {e}")
    print("Hint: If another attribute is missing, add it to the MockFC namedtuple.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

--- Pre-Model Decode Processing (v3) ---
AttributeError: 'MockFC' object has no attribute 'per_core_batch_size'
Hint: If another attribute is missing, add it to the MockFC namedtuple.


## Stacked Transformersの構築と初期化 (Model Initialization)

設定オブジェクト (`config`) に基づき、多層Transformerモデルの構造を定義し、そのパラメータを初期化します。

### 1. 処理の目的 (Purpose)
* **アーキテクチャの定義:** 層の数（Layers）、ヘッドの数（Heads）、モデルの次元（Hidden Dimension）など、ニューラルネットワークの形状を決定します。
* **パラメータ初期化 (Initialization):** 乱数キー (`key`) を使用して、重み（Weights）とバイアス（Biases）に初期値を割り当てます。JAX/Flaxでは、このステップでモデルの変数が確定します。

[Image of Transformer architecture stack]

### 2. 引数の定義 (Arguments)
* **`config`**: モデルのハイパーパラメータを保持する設定オブジェクト。`StackedTransformersConfig` クラス（またはそれに準ずる構造体）のインスタンスが必要です。
* **`key`**: JAXの乱数生成キー (`PRNGKey`)。再現可能な初期化のために使用されます。

In [22]:
# ---------------------------------------------------------------------------
# Transformerモデルの構築と初期化
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

import jax
import jax.numpy as jnp
from collections import namedtuple
from timesfm.timesfm_2p5.timesfm_2p5_flax import _create_stacked_transformers

# --- 1. 設定オブジェクトの作成 (Mock Config) ---
# 本来は configs.StackedTransformersConfig を使用しますが、
# 動作確認用に必要な属性を持つ namedtuple (不変・ハッシュ可能) を定義します。
MockTransformerConfig = namedtuple("MockTransformerConfig", [
    "num_layers",       # レイヤー数 (L)
    "model_dim",        # モデル次元 (D)
    "num_heads",        # 注意機構のヘッド数 (H)
    "dropout_rate",     # ドロップアウト率
    "use_qkv_bias",     # QKV射影にバイアスを使用するか
    "intermediate_dim", # FeedForward層の中間次元
    "activation",       # 活性化関数
    "norm_eps"          # LayerNormのイプシロン
])

# 一般的な「Tiny」サイズの設定で初期化
config_mock = MockTransformerConfig(
    num_layers=2,
    model_dim=64,
    num_heads=4,
    dropout_rate=0.1,
    use_qkv_bias=False,
    intermediate_dim=128,
    activation="gelu",
    norm_eps=1e-6
)

# --- 2. 乱数キーの生成 ---
# 初期化用の乱数シード
init_key = jax.random.PRNGKey(42)

# --- 3. モデル構築関数の実行 ---
print("--- Creating Stacked Transformers ---")

try:
    # モデル構造の作成またはパラメータの初期化を行います
    # resultは通常、初期化されたパラメータ(FrozenDict)やモデル定義が返されます
    result = _create_stacked_transformers(
        config=config_mock, # TODO: configs.StackedTransformersConfig
        key=init_key        # TODO: jax.Array
    )
    
    print("Success: Transformers initialized.")
    print(f"Result Type: {type(result)}")
    
    # もし結果がパラメータ辞書(FrozenDict)なら中身のキーを確認
    if hasattr(result, 'keys'):
        print(f"Top-level keys: {result.keys()}")

except TypeError as e:
    print(f"TypeError: {e}")
    print("Hint: The mock config might be missing attributes required by the specific library version.")
except AttributeError as e:
    print(f"AttributeError: {e}")
    print("Hint: Check if 'config' structure matches the library expectation.")

--- Creating Stacked Transformers ---
AttributeError: 'MockTransformerConfig' object has no attribute 'transformer'
Hint: Check if 'config' structure matches the library expectation.


## 分位点交差の修正 (Fix Quantile Crossing)

確率的予測（Probabilistic Forecasting）の結果において、分位点間の順序関係が逆転している箇所を修正します。

### 1. 処理の目的 (Purpose)
* **論理的整合性の保証:** 分位点予測では、定義上 $Q_{10} \le Q_{50} \le Q_{90}$ のような大小関係が成立していなければなりません。しかし、モデルが各分位点を独立に回帰する場合など、数値計算上でこの順序が崩れる（Crossing）ことがあります。
* **分布の妥当性:** 逆転したままでは確率分布として解釈できないため、これを修正して有効な信頼区間を構築します。



### 2. アルゴリズム (Algorithm)
最も単純かつ効果的な方法は、各タイムステップごとに予測値を**昇順にソート（Sort）**することです。これにより、順序制約が強制的に満たされます。

### 3. 引数の定義 (Arguments)
* **`full_forecast`**: モデルが出力した予測値の配列。形状は通常 `(Batch, Horizon, Quantiles)` です。

In [23]:
# ---------------------------------------------------------------------------
# 分位点交差の修正処理
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

import jax.numpy as jnp
from timesfm.timesfm_2p5.timesfm_2p5_flax import _fix_quantile_crossing_fn

# --- 1. 入力データの準備 (Input Preparation) ---
# テスト用に、分位点の順序が逆転している(交差している)データを作成します。
# Shape: (Batch=1, Horizon=2, Quantiles=3)
# 例: [高い値, 中間の値, 低い値] -> 本来は [低, 中, 高] であるべき
input_forecast = jnp.array([
    [
        [0.9, 0.5, 0.1],  # ケース1: 完全逆転 (Crossing発生)
        [0.2, 0.8, 0.5]   # ケース2: 部分的逆転
    ]
])

print("--- Before Fix (Crossing Data) ---")
print(input_forecast)

# --- 2. 修正関数の実行 (Execution) ---
# NOTE: 前のステップで生成された予測結果がある場合はそれを使用してください
# result_forecast = _fix_quantile_crossing_fn(full_forecast=final_forecast_result[1])

result_fixed = _fix_quantile_crossing_fn(
    full_forecast=input_forecast  # TODO: Float[Array, 'b p q']
)

# --- 3. 結果の確認 (Meta-check) ---
print("\n--- After Fix (Sorted Data) ---")
print(result_fixed)

# 検証: ソートされているか確認
is_sorted = (jnp.diff(result_fixed, axis=-1) >= 0).all()
if is_sorted:
    print("\nSuccess: All quantiles are correctly sorted.")
else:
    print("\nWarning: Quantiles are still crossing.")

--- Before Fix (Crossing Data) ---
[[[0.9 0.5 0.1]
  [0.2 0.8 0.5]]]

--- After Fix (Sorted Data) ---
[[[0.9 0.1 0.1]
  [0.2 0.5 0.5]]]



## 分位点の反転処理 (Flipping Quantiles)

反転入力（$-x$）に対するモデルの予測結果である分位点の順序を逆転させます。

### 1. 処理の目的 (Purpose)
* **反転アンサンブルの整合性:** TimesFMは精度向上のため、元の系列 $x$ と、符号を反転させた系列 $-x$ の両方を予測し、その結果を平均します。
* **分位点の対応関係:** データの符号を反転すると、大小関係が逆転します。例えば、負のデータにおける「上位10%（0.9分位点）」は、元の正のデータにおける「下位10%（0.1分位点）」に相当します。
    * 数式的には: $P(-Y \le q) = \tau \iff P(Y \ge -q) = \tau \iff P(Y \le -q) = 1 - \tau$
    * つまり、反転データの $\tau$ 分位点は、元データの $1-\tau$ 分位点に対応します。

[Image of probability distribution flip quantile]

### 2. アルゴリズム (Algorithm)
入力された分位点配列の**最後の次元（Quantile dimension）を逆順**に並べ替えます。
例: `[0.1, 0.5, 0.9]` $\rightarrow$ `[0.9, 0.5, 0.1]`

### 3. 引数の定義 (Arguments)
* **`x`**: 反転入力に対する予測分位点配列。形状は `(Batch, Horizon, Quantiles)` です。

In [24]:
# ---------------------------------------------------------------------------
# 分位点配列の反転実行
# File: /mnt/e/env/ts/lib_ana/src/model/timesfm/nb/timesfmV6.ipynb
# ---------------------------------------------------------------------------

import jax.numpy as jnp
from timesfm.timesfm_2p5.timesfm_2p5_flax import _flip_quantile_fn

# --- 1. 入力データの準備 (Input Preparation) ---
# 反転入力(-x)から得られた予測分位点と仮定します。
# Shape: (Batch=1, Horizon=1, Quantiles=3)
# 例: 負の世界での [10%点, 50%点, 90%点]
flipped_quantiles_input = jnp.array([
    [[ -10.0, -5.0, -1.0 ]] 
])

print("--- Before Flip Function ---")
print(f"Input Quantiles: {flipped_quantiles_input}")

# --- 2. 関数の実行 (Execution) ---
# 配列の順序を反転させます (Reverse)
result_flipped = _flip_quantile_fn(
    x=flipped_quantiles_input  # TODO: Float[Array, '... q']
)

# --- 3. 結果の確認 (Meta-check) ---
print("\n--- After Flip Function ---")
print(f"Result Quantiles: {result_flipped}")

# 検証: 最初の要素が最後の要素に入れ替わっているか
input_first = flipped_quantiles_input[..., 0]
result_last = result_flipped[..., -1]

if jnp.allclose(input_first, result_last):
    print("\nSuccess: Quantile order has been correctly reversed.")
    print("Explanation: The 10% quantile of -X corresponds to the 90% quantile of X.")
else:
    print("\nWarning: Flip operation did not work as expected.")

--- Before Flip Function ---
Input Quantiles: [[[-10.  -5.  -1.]]]

--- After Flip Function ---
Result Quantiles: [[[-10.  -1.  -5.]]]



## 反転不変性の適用 (Forcing Flip Invariance)

符号を反転させた入力データ（$-x$）から得られたモデル出力に対し、再度符号反転（$-1$ を掛ける）を行い、元のデータ空間（$x$）と整合する値に戻します。

### 1. 処理の目的 (Purpose)
* **二重否定による復元 (Double Negation):** TimesFMの反転アンサンブル戦略では、以下のロジックで予測を統合します。
    1.  **入力反転:** $x \rightarrow -x$
    2.  **推論:** $M(-x) \approx -y$ （モデルは「反転した世界」での未来を予測する）
    3.  **出力反転:** $-(-y) = y$ （予測値を元の世界に戻す）
* **不変性の保証:** この処理により、モデルが「入力の符号が変われば、出力の符号も変わる」という物理的な対称性を満たすことを強制します。

[Image of negative sign inversion math concept]

### 2. 引数の定義 (Arguments)
すべての引数は、反転入力（$-x$）に対するモデルの生出力です。
* **`flipped_pf_outputs`**: 点予測値（Point Forecasts）。
* **`flipped_quantile_spreads`**: 分位点の広がり（Spreads）。※実装により、値そのものか広がりかの定義が異なりますが、ここでは符号反転が必要な成分として扱います。
* **`flipped_ar_outputs`**: 自己回帰成分（Auto-Regressive Outputs）。

In [26]:
import jax.numpy as jnp
from timesfm.flax.transformer import make_attn_mask

# 例: バッチ2、クエリ長128、KV長128（自己注意）を想定
b = 2
query_length = 128
kv_length = 128

# 例: 左パディング等で「先頭から何個のKVトークンを完全に無効化するか」
#   バッチ0は0個、バッチ1は10個ぶんを完全マスク
num_all_masked_kv = jnp.array([0, 10], dtype=jnp.int32)

# 例: 生成（デコード）でキャッシュを使う場合のクエリ位置オフセット（ここでは0）
query_index_offset = 0

attn_mask = make_attn_mask(
    query_length=query_length,
    num_all_masked_kv=num_all_masked_kv,
    query_index_offset=query_index_offset,
    kv_length=kv_length,
)

print(attn_mask.shape, attn_mask.dtype)


IndexError: Too many indices: array is 0-dimensional, but 1 were indexed

### `revin`（RevIN: Reversible Instance Normalization / 可逆インスタンス正規化）
`x` を平均 `mu` と標準偏差 `sigma` で正規化（normalize）または逆変換（denormalize）します。

- `reverse=False`：正規化（おおむね `(x - mu) / sigma`）
- `reverse=True`：逆変換（おおむね `x * sigma + mu`）

`mu` と `sigma` は通常、各系列（各サンプル）ごとに時系列方向などで計算した統計量を入れます。


In [27]:
import jax.numpy as jnp
from timesfm.flax.util import revin

# 例: (batch, time) の時系列
x = jnp.arange(2 * 8, dtype=jnp.float32).reshape(2, 8)

# 例: 時間方向で平均・標準偏差（keepdims=Trueでブロードキャストしやすく）
mu = x.mean(axis=-1, keepdims=True)
sigma = x.std(axis=-1, keepdims=True) + 1e-6  # 0割り防止

x_norm = revin(x=x, mu=mu, sigma=sigma, reverse=False)
x_rec = revin(x=x_norm, mu=mu, sigma=sigma, reverse=True)

print("x:\n", x)
print("x_norm:\n", x_norm)
print("reconstructed (should be close):\n", x_rec)


x:
 [[ 0.  1.  2.  3.  4.  5.  6.  7.]
 [ 8.  9. 10. 11. 12. 13. 14. 15.]]
x_norm:
 [[-1.5275246  -1.091089   -0.6546534  -0.21821779  0.21821779  0.6546534
   1.091089    1.5275246 ]
 [-1.5275246  -1.091089   -0.6546534  -0.21821779  0.21821779  0.6546534
   1.091089    1.5275246 ]]
reconstructed (should be close):
 [[-6.6407267e-08  9.9999994e-01  2.0000000e+00  3.0000000e+00
   4.0000000e+00  5.0000000e+00  6.0000000e+00  7.0000000e+00]
 [ 8.0000000e+00  9.0000000e+00  1.0000000e+01  1.1000000e+01
   1.2000000e+01  1.3000000e+01  1.4000000e+01  1.5000000e+01]]


### `_force_flip_invariance_fn`（符号反転に対する整合性付与：推定）
TimesFM-2.5 の内部処理で、入力系列の **符号反転（flip: x→-x）** に対して予測の整合性を保つための補正を行う関数…と推定されます。

典型的には、
- 点予測（point forecast）は符号が反転（-を掛ける）
- 分位点の“広がり”（quantile spreads）は非負量なので反転せず、順序だけ入れ替える…等

ただし、これは一般論であり、正確には `timesfm_2p5_flax.py` の実装を参照してください。


In [28]:
import jax.numpy as jnp
from timesfm.timesfm_2p5.timesfm_2p5_flax import _force_flip_invariance_fn

# ※形状は実装依存。ここでは「ありがちな」形を仮置きします。
b = 2
horizon = 128
num_q = 9  # 例: 分位点関連の次元（仮）

flipped_pf_outputs = jnp.zeros((b, horizon), dtype=jnp.float32)
flipped_quantile_spreads = jnp.ones((b, horizon, num_q), dtype=jnp.float32)
flipped_ar_outputs = jnp.zeros((b, horizon), dtype=jnp.float32)

result = _force_flip_invariance_fn(
    flipped_pf_outputs=flipped_pf_outputs,
    flipped_quantile_spreads=flipped_quantile_spreads,
    flipped_ar_outputs=flipped_ar_outputs,
)

# 戻り値の型/構造は実装依存なので、まずはprintして確認
print(type(result))
print(result)


IndexError: Too many indices: array is 1-dimensional, but 2 were indexed

In [29]:
import inspect
from timesfm.flax.transformer import make_attn_mask
from timesfm.flax.util import revin
from timesfm.timesfm_2p5.timesfm_2p5_flax import _force_flip_invariance_fn

print(inspect.signature(make_attn_mask))
print(inspect.getsource(make_attn_mask))

print(inspect.signature(revin))
print(inspect.getsource(revin))

print(inspect.signature(_force_flip_invariance_fn))
print(inspect.getsource(_force_flip_invariance_fn))


(query_length: int, num_all_masked_kv: jaxtyping.Integer[jaxlib._jax.Array, 'b'], query_index_offset: jaxtyping.Integer[jaxlib._jax.Array, 'b'] | None = None, kv_length: int = 0) -> jaxtyping.Bool[jaxlib._jax.Array, 'b 1 q n']
@functools.partial(
  jax.jit,
  static_argnames=("query_length", "kv_length"),
)
def make_attn_mask(
  query_length: int,
  num_all_masked_kv: Integer[Array, "b"],
  query_index_offset: Integer[Array, "b"] | None = None,
  kv_length: int = 0,
) -> Bool[Array, "b 1 q n"]:
  """Makes attention mask."""

  if kv_length == 0:
    kv_length = query_length

  q_index = jnp.arange(query_length)[None, None, :, None]
  if query_index_offset is not None:
    q_index += query_index_offset[:, None, None, None]
  kv_index = jnp.arange(kv_length)[None, None, None, :]
  return jnp.logical_and(
    q_index >= kv_index,
    kv_index >= num_all_masked_kv[:, None, None, None],
  )

(x: jaxtyping.Float[jaxlib._jax.Array, 'b ...'], mu: jaxtyping.Float[jaxlib._jax.Array, 'b ...'], si

In [30]:
# Auto-generated call stub (CLE V6)
# NOTE: TODO を埋めた例（形状は README の出力例に合わせた“典型”）

import jax.numpy as jnp
from timesfm.timesfm_2p5.timesfm_2p5_flax import _use_continuous_quantile_head_fn

# 例: バッチサイズと最大予測長（horizon）
b = 2
max_horizon = 256

# full_forecast: 点予測（平均/期待値に相当）を想定
#   典型: (b, max_horizon)
full_forecast = jnp.zeros((b, max_horizon), dtype=jnp.float32)

# quantile_spreads: 平均から各分位点へのオフセット（10%〜90%で9本）を想定
#   典型: (b, max_horizon, 9)
quantile_spreads = jnp.zeros((b, max_horizon, 9), dtype=jnp.float32)

result = _use_continuous_quantile_head_fn(
    full_forecast=full_forecast,
    quantile_spreads=quantile_spreads,
    max_horizon=max_horizon,
)

result


IndexError: Too many indices: array is 2-dimensional, but 3 were indexed

In [32]:
# Auto-generated call stub (CLE V6)

# NOTE: TODO のところを埋めてください

from timesfm.timesfm_2p5.timesfm_2p5_flax import try_gc

result = try_gc(
)
result

In [33]:
import sys
from timesfm.configs import (
    ForecastConfig,
    RandomFourierFeaturesConfig,
    ResidualBlockConfig,
    TransformerConfig,
    StackedTransformersConfig
)

def run_config_demo():
    print("=== TimesFM Config Setup Demo ===")

    # ---------------------------------------------------------
    # 1. ForecastConfig: 予測タスクの設定
    # ---------------------------------------------------------
    # コンテキスト長512, 予測ホライゾン128という一般的な設定
    forecast_config = ForecastConfig(
        max_context=512,
        max_horizon=128,
        normalize_inputs=True,
        window_size=1024,  # context + horizon + margin
        per_core_batch_size=32,
        use_continuous_quantile_head=False,
        force_flip_invariance=True,  # 上下反転した波形でも同じロジックが通用するようにする
        infer_is_positive=False,     # 売上など負の値がないデータならTrue
        fix_quantile_crossing=True,  # 分位点の順序逆転（50%値 > 90%値など）を防ぐ
        return_backcast=False
    )
    print(f"\n[ForecastConfig]\n Context: {forecast_config.max_context}, Horizon: {forecast_config.max_horizon}")

    # ---------------------------------------------------------
    # 2. RandomFourierFeaturesConfig: RFF層の設定
    # ---------------------------------------------------------
    # 時系列の特徴抽出に使われるランダムフーリエ特徴量
    rff_config = RandomFourierFeaturesConfig(
        input_dims=1,           # 単変量時系列なら1
        output_dims=64,         # 特徴空間への射影次元
        projection_stddev=1.0,  # 重み初期化の標準偏差
        use_bias=True
    )
    print(f"\n[RandomFourierFeaturesConfig]\n Input: {rff_config.input_dims} -> Output: {rff_config.output_dims}")

    # ---------------------------------------------------------
    # 3. ResidualBlockConfig: 残差ブロックの設定
    # ---------------------------------------------------------
    # トークナイザーや出力層として使われるMLPブロック
    res_block_config = ResidualBlockConfig(
        input_dims=64,          # RFFの出力などを受け取る
        hidden_dims=1280,       # モデル次元へ拡張
        output_dims=1280,
        use_bias=True,
        activation='swish'      # Swish (SiLU) 活性化関数
    )
    print(f"\n[ResidualBlockConfig]\n Dims: {res_block_config.input_dims} -> {res_block_config.hidden_dims} -> {res_block_config.output_dims}")
    print(f" Activation: {res_block_config.activation}")

    # ---------------------------------------------------------
    # 4. TransformerConfig: 単一Transformer層の設定
    # ---------------------------------------------------------
    # TimesFM 2.5 200Mモデルに近い設定
    # - RMSNorm採用
    # - RoPE採用
    # - Swish活性化関数
    tf_layer_config = TransformerConfig(
        model_dims=1280,
        hidden_dims=1280,       # FFNの隠れ層サイズ
        num_heads=16,           # 1280 / 16 = 80次元/ヘッド
        attention_norm='rms',   # RMSNorm
        feedforward_norm='rms',
        qk_norm='rms',          # Query/Keyの正規化（学習安定化のため）
        use_bias=False,         # 最近のLLM/Transformerはバイアスを省く傾向
        use_rotary_position_embeddings=True,
        ff_activation='swish',
        fuse_qkv=False
    )
    print(f"\n[TransformerConfig (Single Layer)]\n Model Dims: {tf_layer_config.model_dims}, Heads: {tf_layer_config.num_heads}")
    print(f" Norm Type: {tf_layer_config.attention_norm}, RoPE: {tf_layer_config.use_rotary_position_embeddings}")

    # ---------------------------------------------------------
    # 5. StackedTransformersConfig: 積み上げ設定
    # ---------------------------------------------------------
    # 上記の層を何層積み重ねるか定義
    stacked_config = StackedTransformersConfig(
        num_layers=20,          # 20層積み上げ
        transformer=tf_layer_config
    )
    print(f"\n[StackedTransformersConfig]\n Total Layers: {stacked_config.num_layers}")
    print(" --> Configuration objects are ready for model initialization.")

if __name__ == "__main__":
    run_config_demo()

=== TimesFM Config Setup Demo ===

[ForecastConfig]
 Context: 512, Horizon: 128

[RandomFourierFeaturesConfig]
 Input: 1 -> Output: 64

[ResidualBlockConfig]
 Dims: 64 -> 1280 -> 1280
 Activation: swish

[TransformerConfig (Single Layer)]
 Model Dims: 1280, Heads: 16
 Norm Type: rms, RoPE: True

[StackedTransformersConfig]
 Total Layers: 20
 --> Configuration objects are ready for model initialization.


In [34]:
import itertools
from timesfm.configs import (
    ForecastConfig,
    TransformerConfig,
    StackedTransformersConfig,
    ResidualBlockConfig,
    RandomFourierFeaturesConfig
)

def format_size(num):
    """パラメータ数を読みやすくフォーマット"""
    if num >= 1_000_000_000:
        return f"{num / 1_000_000_000:.2f}B"
    elif num >= 1_000_000:
        return f"{num / 1_000_000:.2f}M"
    else:
        return f"{num / 1_000:.2f}K"

def estimate_params(t_config, s_config):
    """
    Transformerのおおよそのパラメータ数を概算する関数
    (注意: 正確な実装依存のバイアス等は無視した簡易計算)
    """
    d_model = t_config.model_dims
    d_ff = t_config.hidden_dims
    n_layers = s_config.num_layers
    
    # 1層あたりの概算
    # Attention: 4 * d_model^2 (Q,K,V,Output)
    # FFN: 2 * d_model * d_ff (Up, Down)
    attn_params = 4 * (d_model ** 2)
    ffn_params = 2 * (d_model * d_ff)
    
    layer_params = attn_params + ffn_params
    total_params = layer_params * n_layers
    
    return total_params

def exploratory_config_generation():
    print("=== TimesFM Configuration Exploratory Verification ===\n")

    # 1. 検証したいハイパーパラメータの候補を定義（探索空間）
    search_space = {
        'context_horizon': [(512, 128), (2048, 512)], # (Context, Horizon)
        'model_size': [
            {'d_model': 512, 'heads': 8, 'layers': 6, 'name': 'Small (Debug)'},
            {'d_model': 1280, 'heads': 16, 'layers': 20, 'name': 'Base (200M)'},
            {'d_model': 2560, 'heads': 20, 'layers': 32, 'name': 'Large (Production)'},
        ],
        'norms': ['rms', 'layer'],
        'use_bias': [True, False]
    }

    # 2. 組み合わせを生成して検証
    print(f"{'Config Name':<20} | {'Context':<8} | {'Horizon':<8} | {'Norm':<6} | {'Bias':<6} | {'Est. Params':<10}")
    print("-" * 80)

    # itertools.productで全組み合わせを探索
    for (ctx, hor), size_conf, norm, bias in itertools.product(
        search_space['context_horizon'],
        search_space['model_size'],
        search_space['norms'],
        search_space['use_bias']
    ):
        # 設定オブジェクトの生成検証
        try:
            # Forecast Config
            f_conf = ForecastConfig(
                max_context=ctx,
                max_horizon=hor,
                normalize_inputs=True,
                window_size=ctx + hor + 128,
                per_core_batch_size=32
            )
            
            # Transformer Config
            t_conf = TransformerConfig(
                model_dims=size_conf['d_model'],
                hidden_dims=size_conf['d_model'], # TimesFM style
                num_heads=size_conf['heads'],
                attention_norm=norm,
                feedforward_norm=norm,
                qk_norm=norm,
                use_bias=bias,
                use_rotary_position_embeddings=True,
                ff_activation='swish',
                fuse_qkv=True
            )
            
            # Stacked Config
            s_conf = StackedTransformersConfig(
                num_layers=size_conf['layers'],
                transformer=t_conf
            )
            
            # パラメータ数概算
            param_count = estimate_params(t_conf, s_conf)
            
            # 結果表示
            print(f"{size_conf['name']:<20} | {ctx:<8} | {hor:<8} | {norm:<6} | {str(bias):<6} | {format_size(param_count):<10}")
            
        except Exception as e:
            print(f"Invalid Config Combination: {e}")

    print("\n=== Verification Complete ===")
    print("ヒント: 'Small'構成は手元のCPUでのデバッグや単体テストに最適です。")
    print("ヒント: 'Base'構成が配布されているTimesFM-2.5のチェックポイントと互換性があります。")

if __name__ == "__main__":
    exploratory_config_generation()

=== TimesFM Configuration Exploratory Verification ===

Config Name          | Context  | Horizon  | Norm   | Bias   | Est. Params
--------------------------------------------------------------------------------
Small (Debug)        | 512      | 128      | rms    | True   | 9.44M     
Small (Debug)        | 512      | 128      | rms    | False  | 9.44M     
Small (Debug)        | 512      | 128      | layer  | True   | 9.44M     
Small (Debug)        | 512      | 128      | layer  | False  | 9.44M     
Base (200M)          | 512      | 128      | rms    | True   | 196.61M   
Base (200M)          | 512      | 128      | rms    | False  | 196.61M   
Base (200M)          | 512      | 128      | layer  | True   | 196.61M   
Base (200M)          | 512      | 128      | layer  | False  | 196.61M   
Large (Production)   | 512      | 128      | rms    | True   | 1.26B     
Large (Production)   | 512      | 128      | rms    | False  | 1.26B     
Large (Production)   | 512      | 128      | lay

In [37]:
import jax
import jax.numpy as jnp
from timesfm.flax.transformer import make_attn_mask

def demo_make_attn_mask_fixed():
    print("=== TimesFM Attention Mask Demo (Fixed) ===\n")

    # 設定: バッチサイズ=2, シーケンス長=4
    batch_size = 2
    seq_len = 4
    
    # 修正1: num_all_masked_kv は (Batch,) の形状を持つ配列にする
    # ここでは「パディングなし（全てのトークンが有効）」として 0 を指定
    num_all_masked_kv = jnp.zeros((batch_size,), dtype=jnp.int32)
    
    # 修正2: query_index_offset も同様に配列で指定（オフセットなしなら 0）
    query_index_offset = jnp.zeros((batch_size,), dtype=jnp.int32)

    try:
        mask = make_attn_mask(
            query_length=seq_len, 
            num_all_masked_kv=num_all_masked_kv,  # 必須: JAX配列
            query_index_offset=query_index_offset, # 推奨: JAX配列
            kv_length=seq_len
        )
        
        print(f"Success! Mask shape: {mask.shape}")
        # 期待される形状: (Batch, 1, Query, Key) -> (2, 1, 4, 4)
        
        print("\n--- Mask Content (Batch 0) ---")
        # 1 = Attention可能, 0 = Mask
        # 下三角行列（未来の情報をマスク）になっているか確認
        print(mask[0, 0, :, :])
        
    except Exception as e:
        print(f"Error occurred: {e}")

if __name__ == "__main__":
    demo_make_attn_mask_fixed()

=== TimesFM Attention Mask Demo (Fixed) ===

Success! Mask shape: (2, 1, 4, 4)

--- Mask Content (Batch 0) ---
[[ True False False False]
 [ True  True False False]
 [ True  True  True False]
 [ True  True  True  True]]


In [ ]:
import jax
import jax.numpy as jnp
from flax import nnx

# --- 修正版インポート ---
from timesfm.flax.util import revin  # revinはutilのまま
from timesfm.flax.transformer import make_attn_mask, Transformer, RotaryPositionalEmbedding # make_attn_maskはこちら
from timesfm.configs import TransformerConfig

def demo_flax_components_fixed():
    print("=== TimesFM Flax Components Demo (Fixed) ===\n")

    # 1. RevIN Demo
    # (コードは前回と同じですが、インポートが正しければ動作します)
    x = jnp.array([[[100.0], [102.0], [105.0]], [[1.0], [2.0], [1.0]]])
    mu = jnp.mean(x, axis=1, keepdims=True)
    sigma = jnp.std(x, axis=1, keepdims=True) + 1e-6
    
    # revinのテスト
    x_norm = revin(x=x, mu=mu, sigma=sigma, reverse=False)
    print(f"RevIN Normalized shape: {x_norm.shape}")

    # 2. Attention Mask Demo
    # 正しいモジュールからインポートされた make_attn_mask を使用
    try:
        mask = make_attn_mask(
            query_length=4, 
            num_all_masked_kv=None, # 必須引数として定義されている場合があるためNoneまたは適切な値を指定
            query_index_offset=None,
            kv_length=4
        )
        print(f"Attention Mask created: {mask.shape}")
        # (Batch, 1, Query, Key) のような形状になるはずです
        print(mask[0, 0, :, :]) 
        
    except TypeError as e:
        # 引数が足りない場合のフォールバック（バージョンによって引数が異なる可能性があります）
        print(f"Mask creation check: {e}")
        # 簡易版呼び出し（スタブの定義に基づく）
        mask = make_attn_mask(query_length=4, kv_length=4)
        print("Mask created with minimal args.")

if __name__ == "__main__":
    demo_flax_components_fixed()